<a href="https://colab.research.google.com/github/DojunPark/Research_Undergraduate_Student_Program/blob/master/%5Bmachinetranslation%5D%2005_comparing_data_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Korean-English dataset

In [19]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news800k.csv')

print(df.shape)
df.head()

(801387, 2)


,kor,eng
0,스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.,Skinner's reward is mostly eye-watering.
1,심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.,Even some problems can be predicted.
2,오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.,Only God will exactly know why.
3,중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.,Businesses should not overlook China's dispute.
4,박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.,Slow-beating songs often float over time.


In [ ]:
!pip install konlpy
! sudo apt-get install curl git
! bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
!python -m spacy download en

In [5]:
from konlpy.tag import Mecab
import spacy

mecab = Mecab()
spacy_en = spacy.load('en')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=mecab.morphs, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

In [18]:
from sklearn.model_selection import train_test_split

train_data1, test_data1 = train_test_split(df, test_size=0.2, random_state=1)
train_data1, valid_data1 = train_test_split(train_data1, test_size=0.25, random_state=1)

In [21]:
print('number of train_data1: {}, {:.1f}%'.format(len(train_data1), len(train_data1)/(len(train_data1)+len(test_data1)+len(valid_data1))))
print('number of test_data1: {}, {:.1f}%'.format(len(test_data1), len(test_data1)/(len(train_data1)+len(test_data1)+len(valid_data1))))
print('number of valid_data1: {}, {:.1f}%'.format(len(valid_data1), len(valid_data1)/(len(train_data1)+len(test_data1)+len(valid_data1))))

number of train_data1: 480831, 0.6%
number of test_data1: 160278, 0.2%
number of valid_data1: 160278, 0.2%


In [26]:
train_data1.to_csv('/content/drive/My Drive/Colab Notebooks/train1.csv', sep=',')
valid_data1.to_csv('/content/drive/My Drive/Colab Notebooks/valid1.csv', sep=',')
test_data1.to_csv('/content/drive/My Drive/Colab Notebooks/test1.csv', sep=',')

In [28]:
from torchtext.data import Field, TabularDataset, BucketIterator

fields = {'kor': ('src', korean), 'eng':('trg', english)}

train_data1, valid_data1, test_data1 = TabularDataset.splits(
                                                    path = '/content/drive/My Drive/Colab Notebooks',
                                                    train = 'train1.csv',
                                                    validation = 'valid1.csv',
                                                    test = 'test1.csv',
                                                    format = 'csv',
                                                    fields = fields)

In [36]:
korean.build_vocab(train_data1, min_freq=2)  # ???
english.build_vocab(train_data1, min_freq=2)

# German-English dataset

In [ ]:
!python -m spacy download de

In [13]:
import spacy

spacy_de = spacy.load('de')

In [15]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

print(tokenize_de('hallo guten tag mein name ist dojun.'))

['hallo', 'guten', 'tag', 'mein', 'name', 'ist', 'dojun', '.']


In [32]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

german = Field(tokenize=tokenize_de, lower=True, init_token='<sos>', eos_token='<eos>')
english2 = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

In [33]:
train_data2, valid_data2, test_data2 = Multi30k.splits(exts=('.de', '.en'), fields=(german, english2))

In [35]:
german.build_vocab(train_data2, min_freq=2)  
english2.build_vocab(train_data2, min_freq=2)

# Comparing Korean-English dataset to German-English dataset

## train_data

In [43]:
train_data1

In [37]:
train_data1.__dict__

{'examples': [<torchtext.data.example.Example at 0x7fc49f7c7898>,
  ...],
 'fields': {'src': <torchtext.data.field.Field at 0x7fc4827f4588>,
  'trg': <torchtext.data.field.Field at 0x7fc4827f45f8>}}

In [38]:
train_data2.__dict__

{'examples': [<torchtext.data.example.Example at 0x7fc2f7cf1358>,
  ...],
 'fields': {'src': <torchtext.data.field.Field at 0x7fc2f7cee438>,
  'trg': <torchtext.data.field.Field at 0x7fc2f7cee6a0>}}

- Korean-English dataset

In [ ]:
{'examples': [<torchtext.data.example.Example at 0x7fc49f7c7898>,
            ...
            <torchtext.data.example.Example at 0x7fc401091358>,
            ...],
'fields': {'src': <torchtext.data.field.Field at 0x7fc4827f4588>,
'trg': <torchtext.data.field.Field at 0x7fc4827f45f8>}}

- German-English dataset

In [ ]:
{'examples': [<torchtext.data.example.Example at 0x7fc41eb84518>,
            ...
            <torchtext.data.example.Example at 0x7fc48b1bcd68>,
            ...],
'fields': {'src': <torchtext.data.field.Field at 0x7fc4268c2e10>,
'trg': <torchtext.data.field.Field at 0x7fc4827f45f8>}}

## train_data[0]

In [40]:
train_data1[0]

In [45]:
print(train_data1[0].__dict__)

{'src': ['지난', '세르비아', '와', '평가전', '등', 'a', '매치', '출전', '기록', '이', '7', '경기', '에', '불과', '하', '지만', '‘', '대구', '데', '헤', '아', '’', '라는', '별명', '답', '게', '안정', '적', '인', '선방', '으로', '믿', '음', '을', '얻', '었', '다', '.'], 'trg': ['he', '/', 'she', 'has', 'only', 'seven', 'a', '-', 'match', 'records', ',', 'including', 'the', 'last', 'match', 'against', 'serbia', ',', 'but', 'he', '/', 'she', 'has', 'earned', 'his', 'trust', 'by', 'being', 'nicknamed', '"', 'daegu', 'de', 'gea', '"', 'as', 'a', 'stable', 'defense', '.']}


In [46]:
print(train_data2[0].__dict__)

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


- Korean to English

In [ ]:
{'src': ['지난', ..., '얻', '었', '다', '.'], 'trg': ['he', ..., 'a', 'stable', 'defense', '.']}

- German to English

In [ ]:
{'src': ['zwei', ..., 'vieler', 'büsche', '.'], 'trg': ['two', ..., 'many', 'bushes', '.']}

## dir(train_data)

In [49]:
dir(train_data1)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'download',
 'examples',
 'fields',
 'filter_examples',
 'sort_key',
 'split',
 'splits']

In [50]:
dir(train_data2)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'dirname',
 'download',
 'examples',
 'fields',
 'filter_examples',
 'name',
 'sort_key',
 'split',
 'splits',
 'urls']

In [51]:
print(len(dir(train_data1)))
print(len(dir(train_data2)))

38
41


In [55]:
len(train_data1)

480831

In [56]:
len(train_data2)

29000

In [57]:
i = 1
for data in train_data1:
    print(data.__dict__)
    i += 1
    if i > 10:
        break

{'src': ['지난', '세르비아', '와', '평가전', '등', 'a', '매치', '출전', '기록', '이', '7', '경기', '에', '불과', '하', '지만', '‘', '대구', '데', '헤', '아', '’', '라는', '별명', '답', '게', '안정', '적', '인', '선방', '으로', '믿', '음', '을', '얻', '었', '다', '.'], 'trg': ['he', '/', 'she', 'has', 'only', 'seven', 'a', '-', 'match', 'records', ',', 'including', 'the', 'last', 'match', 'against', 'serbia', ',', 'but', 'he', '/', 'she', 'has', 'earned', 'his', 'trust', 'by', 'being', 'nicknamed', '"', 'daegu', 'de', 'gea', '"', 'as', 'a', 'stable', 'defense', '.']}
{'src': ['범인', '을', '꼭', '잡', '겠', '다고', '다짐', '한', '오', '써니', '에게', '태황태후', '를', '보필', '했', '던', '홍', '팀장', '이', '몰래', '찾아왔', '고', ',', '오', '써니', '는', '태황태후', '가', '죽', '기', '전', '황실', '의', '비리', '증거', '를', '모두', '담', '은', '봉투', '를', '홍', '팀장', '에게', '전달', '했', '다는', '사실', '을', '알', '게', '됐', '다', '.'], 'trg': ['team', 'leader', 'hong', 'who', 'had', 'helped', 'empress', 'taehwang', 'secretly', 'visited', 'oh', 'sunny', 'who', 'vowed', 'to', 'catch', 'the', 'criminal', 'a

In [58]:
i = 1
for data in train_data2:
    print(data.__dict__)
    i += 1
    if i > 10:
        break

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}
{'src': ['mehrere', 'männer', 'mit', 'schutzhelmen', 'bedienen', 'ein', 'antriebsradsystem', '.'], 'trg': ['several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']}
{'src': ['ein', 'kleines', 'mädchen', 'klettert', 'in', 'ein', 'spielhaus', 'aus', 'holz', '.'], 'trg': ['a', 'little', 'girl', 'climbing', 'into', 'a', 'wooden', 'playhouse', '.']}
{'src': ['ein', 'mann', 'in', 'einem', 'blauen', 'hemd', 'steht', 'auf', 'einer', 'leiter', 'und', 'putzt', 'ein', 'fenster', '.'], 'trg': ['a', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'standing', 'on', 'a', 'ladder', 'cleaning', 'a', 'window', '.']}
{'src': ['zwei', 'männer', 'stehen', 'am', 'herd', 'und', 'bereiten', 'essen', 'zu', '.'], 'trg': ['two', 'men', 'are', 'at', 'the', 'stove', 'pre